**The learning code I used is the same as below.**

https://www.kaggle.com/awsaf49/tf-petfinder-vit-cls-tpu-train

His notebook is very useful. Please keep that in mind.

I modified only the build_model function.

Several custom functions were used. -> Mish, Gradient Centralization

**tf swin transformer**

https://github.com/rishigami/Swin-Transformer-TF

https://www.kaggle.com/rishigami/tpu-swin-transformer-tensorflow

In [ ]:
import tensorflow as tf

import sys
sys.path.append('../input/swintransformertf')
from swintransformer import SwinTransformer

In [ ]:
class CFG:
    img_size = [384, 384]
    model_name = 'swin_large_384'

# define custom function
**Gradient Centralization TensorFlow**

https://github.com/Rishit-dagli/Gradient-Centralization-TensorFlow

https://keras.io/examples/vision/gradient_centralization/

**custom mish**

https://www.kaggle.com/imokuri/mish-activation-function

In [ ]:
#custom function
class Mish(tf.keras.layers.Activation):
    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

tf.keras.utils.get_custom_objects().update({'Mish': Mish(mish)})

In [ ]:
def get_centralized_gradients(optimizer, loss, params):
    grads = []
    for grad in K.gradients(loss, params):
        grad_len = len(grad.shape)
        if grad_len > 1:
            axis = list(range(grad_len - 1))
            grad -= tf.reduce_mean(grad,
                                   axis=axis,
                                   keep_dims=True)
        grads.append(grad)

    if None in grads:
        raise ValueError('An operation has `None` for gradient. '
                         'Please make sure that all of your ops have a '
                         'gradient defined (i.e. are differentiable). '
                         'Common ops without gradient: '
                         'K.argmax, K.round, K.eval.')
    if hasattr(optimizer, 'clipnorm') and optimizer.clipnorm > 0:
        norm = K.sqrt(sum([K.sum(K.square(g)) for g in grads]))
        grads = [
            tf.keras.optimizers.clip_norm(
                g,
                optimizer.clipnorm,
                norm) for g in grads]
    if hasattr(optimizer, 'clipvalue') and optimizer.clipvalue > 0:
        grads = [K.clip(g, -optimizer.clipvalue, optimizer.clipvalue)
                 for g in grads]
    return grads


def centralized_gradients_for_optimizer(optimizer):
    def get_centralized_gradients_for_optimizer(loss, params):
        return get_centralized_gradients(optimizer, loss, params)

    return get_centralized_gradients_for_optimizer

**If there is an error in the input layer, add the resize layer.**

In [ ]:
def build_model(model_name=CFG.model_name, DIM=CFG.img_size[0], compile_model=True, include_top=False):
    image_input = tf.keras.layers.Input(shape=(DIM,DIM,3))
    #img_adjust_layer = tf.keras.layers.experimental.preprocessing.Resizing(DIM, DIM)
    pretrained_model = SwinTransformer(CFG.model_name,include_top=False, pretrained=True, use_tpu=True)
    base = tf.keras.Sequential([pretrained_model,
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense(64, activation='Mish'),
                                tf.keras.layers.Dropout(0.2),
                                tf.keras.layers.Dense(1, activation='sigmoid')])
    #x = img_adjust_layer(image_input)
    x = base(image_input)
    model = tf.keras.Model(inputs = image_input, outputs = x)
    if compile_model:
        #optimizer
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        optimizer.get_gradients=centralized_gradients_for_optimizer(optimizer) 
        #loss
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01)
        #metric
        rmse = RMSE
        model.compile(optimizer=optimizer,
                      loss=loss,
                      metrics=[rmse])
    return model

In [ ]:
tmp = build_model(CFG.model_name, DIM=CFG.img_size[0], compile_model=False)
tmp.summary()

In [ ]:
tmp.layers[1].summary()